## Introduction to the model registry with Hopsworks
In this notebook we will deploy and serve our previously created model using the Hopsworks registry, we will then use test data to return a prediction

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import hopsworks
 
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

In [ ]:
ms = project.get_model_serving()
try:
    deployment = ms.get_deployment("titanic_deployed")
except:
    deployment = titanic_model.deploy(name="titanic_deployed",
                                   script_file=predictor_script_path,  
                                   model_server="PYTHON", 
                                   serving_tool="KSERVE")

print("Deployment: " + deployment.name)
deployment.describe()

In [ ]:
state = deployment.get_state()

if state.status != "Running":
    deployment.start()
    deployment.describe()
else:
    print("Deployment already running")

In [ ]:
test_data = titanic_model.input_example

data = {"instances" : [test_data]}
res = deployment.predict(data)
print(test_data)
print(le.inverse_transform([res["predictions"][0]]))